In [1]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16		

vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
vgg_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [2]:
x = vgg_model.output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x) # Dropout layer to reduce overfitting
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dense(136, activation='relu')(x) # Softmax for multiclass
transfer_model = tf.keras.Model(inputs=vgg_model.input, outputs=x)

In [3]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=8, verbose=1, mode='max', min_lr=5e-5)
checkpoint = tf.keras.callbacks.ModelCheckpoint('vgg16_finetune.h15', monitor= 'val_accuracy', mode='max', save_best_only=True, verbose=1)

In [4]:
learning_rate= 5e-5
transfer_model.compile(loss="mse", optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), metrics=["accuracy"])

In [5]:
import os
from libs.dp import Dataset

In [6]:
# checkpoint

checkpoint_path = "data/checkpoint_vgg16/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq='epoch')

transfer_model.save_weights(checkpoint_path.format(epoch=0))

In [7]:
IMG_RES = 256
HMS_RES = 64
NUM_LANDMARKS = 68
BATCH_SIZE = 16
DATASET_DIR = '../Data/300W_train/train.csv'

dataset = Dataset(IMG_RES, HMS_RES, NUM_LANDMARKS, DATASET_DIR)
# dataset_generator = 

Train dataset: ../Data/300W_train/train.csv
Train dataset is loaded. Shape: (3837, 2)


In [8]:
import numpy as np

In [9]:
tf_dataset = tf.data.Dataset.from_generator(dataset.gen_vgg16,
                                            output_types=(np.float32, np.float32),
                                            output_shapes=(
                                                [IMG_RES, IMG_RES, 3],
                                                [NUM_LANDMARKS * 2]),
                                            args=()).repeat().batch(BATCH_SIZE)

In [10]:
VAL_DIR = '../Data/300W/eval.csv'
val_dataset = Dataset(IMG_RES, HMS_RES, NUM_LANDMARKS, VAL_DIR)
val_tf_dataset = tf.data.Dataset.from_generator(dataset.gen_vgg16,
                                                output_types=(np.float32, np.float32),
                                                output_shapes=(
                                                    [IMG_RES, IMG_RES, 3],
                                                    [NUM_LANDMARKS * 2]),
                                                args=()).repeat().batch(BATCH_SIZE)


Train dataset: ../Data/300W/eval.csv
Train dataset is loaded. Shape: (600, 2)


In [13]:
hist = transfer_model.fit(tf_dataset.take(240), validation_data=val_tf_dataset.take(19), callbacks=[lr_reduce], epochs=100)

Epoch 1/100
240/240 [==============================] - 284s 1s/step - loss: 0.0659 - accuracy: 0.0789 - val_loss: 0.0186 - val_accuracy: 0.2928
Epoch 2/100
240/240 [==============================] - 279s 1s/step - loss: 0.0244 - accuracy: 0.1042 - val_loss: 0.0101 - val_accuracy: 0.2237
Epoch 3/100
240/240 [==============================] - 279s 1s/step - loss: 0.0168 - accuracy: 0.1237 - val_loss: 0.0109 - val_accuracy: 0.1513
Epoch 4/100
240/240 [==============================] - 277s 1s/step - loss: 0.0148 - accuracy: 0.1385 - val_loss: 0.0101 - val_accuracy: 0.1283
Epoch 5/100
240/240 [==============================] - 281s 1s/step - loss: 0.0135 - accuracy: 0.1474 - val_loss: 0.0092 - val_accuracy: 0.2007
Epoch 6/100
240/240 [==============================] - 278s 1s/step - loss: 0.0128 - accuracy: 0.1487 - val_loss: 0.0088 - val_accuracy: 0.2599
Epoch 7/100
240/240 [==============================] - 277s 1s/step - loss: 0.0122 - accuracy: 0.1651 - val_loss: 0.0085 - val_accuracy: